## Import Libraries

In [ ]:
import sempy_labs as labs
from sempy_labs.tom import connect_semantic_model

## Set Workspace and Dataset Variables

In [ ]:
workspace = 'Workspace ID'
dataset = 'Dataset Name'

## Connect to Semnatic model

In [ ]:
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    for t in tom.model.Tables:
        print(t.Name)

## Rename Objects in SM

In [ ]:
# Rename objects in the semantic model
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    for t in tom.model.Tables:
        t.Name = t.Name.replace('_',' ')

with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    for c in tom.all_columns():
        c.Name = c.Name.replace('_',' ')

In [ ]:
# Function to convert to Snake case
def to_title_case(snake_str):
    return snake_str.title()
# Convert table names to title case
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    for t in tom.model.Tables:
        t.Name = to_title_case(t.Name)

# Convert column names to Snake case
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    for c in tom.all_columns():
        c.Name = to_title_case(c.Name)

In [ ]:
# with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
#     for t in tom.model.Tables:
#         print(t.Name)

## Create Measures

In [ ]:
# with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
#     for t in tom.model.Tables:
#         if t.Name == 'Europe Weather Data':
#             # List of measures to add or update
#             measures = [
#                 {
#                     'name': 'Measure Name',
#                     'expression': "Measure Expression",
#                     'format_string': "#,0.00",
#                     'description': "Measure Description",
#                     'display_folder': "Measure Folder"
#                 }
#                 ]

#             # Add or update measures
#             for measure in measures:
#                 if measure['name'] in [m.Name for m in t.Measures]:
#                     # Update existing measure
#                     existing_measure = t.Measures[measure['name']]
#                     existing_measure.Expression = measure['expression']
#                     existing_measure.FormatString = measure['format_string']
#                     existing_measure.Description = measure['description']
#                     existing_measure.DisplayFolder = measure['display_folder']
#                 else:
#                     # Add new measure
#                     tom.add_measure(
#                         table_name=t.Name, 
#                         measure_name=measure['name'], 
#                         expression=measure['expression'], 
#                         format_string=measure['format_string'],
#                         description=measure['description'],
#                         display_folder=measure['display_folder']
#                     )


In [ ]:
# Rename objects in the semantic model
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    for t in tom.model.Tables:
        if t.Name == 'Europe Weather Data':
            # List of measures to add or update
            measures = [
                            {
                                'name': 'Total Rainfall',
                                'expression': "SUM('Europe Weather Data'[Rain])",
                                'format_string': "#,0.0",
                                'description': "Total amount of Rainfall",
                                'display_folder': "Precipitation Metrics"
                            },
                            {
                                'name': 'Total Snowfall',
                                'expression': "SUM('Europe Weather Data'[Snowfall])",
                                'format_string': "#,0.00",
                                'description': "Total amount of Snowfall",
                                'display_folder': "Precipitation Metrics"
                            }
                        ]

            
            # Add or update measures
            for measure in measures:
                if measure['name'] in [m.Name for m in t.Measures]:
                    # Update existing measure
                    existing_measure = t.Measures[measure['name']]
                    existing_measure.Expression = measure['expression']
                    existing_measure.FormatString = measure['format_string']
                    existing_measure.Description = measure['description']
                    existing_measure.DisplayFolder = measure['display_folder']
                else:
                    # Add new measure
                    tom.add_measure(
                        table_name=t.Name, 
                        measure_name=measure['name'], 
                        expression=measure['expression'], 
                        format_string=measure['format_string'],
                        description=measure['description'],
                        display_folder=measure['display_folder']
                    )

### Delete Measures

In [ ]:
# with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
#     for t in tom.model.Tables:
#         if t.Name == 'Europe Weather Data':
#             # List of measures to delete
#             measures_to_delete = [m for m in t.Measures]
            
#             # Delete each measure
#             for measure in measures_to_delete:
#                 tom.remove_object(measure)

## Create Calculation Groups

In [ ]:
# Define the calculation group
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    
    calculation_group_name = "Time Intelligence"
    precedence=1,
    description="Calculation group for Time Intelligence",
    hidden=False
    calculation_items = [
        {
            "table_name": "Europe Weather Data",
            "name": "Current",
            "expression": "SELECTEDMEASURE()"
        },
        {
            "table_name": "Europe Weather Data",
            "name": "PY",
            "expression": "CALCULATE(SELECTEDMEASURE(), SAMEPERIODLASTYEAR('Date Table'[Year]))"
        },
        {
            "table_name": "Europe Weather Data",
            "name": "YOY",
            "expression": "SELECTEDMEASURE() - CALCULATE(SELECTEDMEASURE(), 'Time Intelligence'[Name] = \"PY\")"
        },
        {
            "table_name": "Europe Weather Data",
            "name": "YOY%",
            "expression": "DIVIDE(CALCULATE(SELECTEDMEASURE(), 'Time Intelligence'[Name]=\"YOY\"), CALCULATE(SELECTEDMEASURE(), 'Time Intelligence'[Name]=\"PY\"))"
        }
    ]
    

    # Retrieve all calculation groups
    all_calculation_groups = tom.all_calculation_groups()

    # Check if the calculation group already exists
    existing_group = next((cg for cg in all_calculation_groups if cg.Name == calculation_group_name), None)

    if existing_group:
        # Update existing calculation group
        existing_group.CalculationItems = calculation_items
        print(f"Updated calculation group: {calculation_group_name}")
    else:
        # Add new calculation group
        calculation_group = {
            "name": calculation_group_name,
            "calculationItems": calculation_items
        }
        tom.add_calculation_group(
            name=calculation_group_name,  
            precedence=1,
            description="DESC ",
            hidden=False
        )

        for item in calculation_items:
            tom.add_calculation_item(
                table_name=calculation_group_name,
                calculation_item_name=item['name'],
                expression=item['expression']
            )
        print(f"Added new calculation group: {calculation_group_name}")
